In [1]:
# Install necessary libraries
%pip install "pybamm[plot,cite]" -q

import pybamm
import matplotlib.pyplot as plt

# Define the battery model
model = pybamm.lithium_ion.DFN(
    {
        "SEI": "solvent-diffusion limited",
        "SEI porosity change": "true",
        "lithium plating": "partially reversible",
        "lithium plating porosity change": "true",
        "particle mechanics": ("swelling and cracking", "swelling only"),
        "SEI on cracks": "true",
        "loss of active material": "stress-driven",
        "calculate discharge energy": "true",
    }
)

# Set model parameters
param = pybamm.ParameterValues("OKane2022")
param.update({
    "SEI solvent diffusivity [m2.s-1]": 2e-21,
    "Dead lithium decay constant [s-1]": 7.5e-5,
})

var_pts = {
    "x_n": 5,  # Negative electrode
    "x_s": 5,  # Separator
    "x_p": 5,  # Positive electrode
    "r_n": 30,  # Negative electrode particles
    "r_p": 30,  # Positive electrode particles
}

# Set the target number of cycles
target_cycle = 1100

print(f"Calculating cycle {target_cycle} ...")

# Define the experimental protocol
exp_k = pybamm.Experiment(
    [
        "Hold at 4.2 V until C/100 (5 minute period)",
        "Rest for 4 hours (5 minute period)",
        "Discharge at 0.1C until 2.5 V (5 minute period)",
        "Charge at 0.3C until 4.2 V (5 minute period)",
        "Hold at 4.2 V until C/100 (5 minute period)",
    ]
    + [
        (
            "Discharge at 1C until 2.5 V",
            "Charge at 0.3C until 4.2 V (5 minute period)",
            "Hold at 4.2 V until C/100 (5 minute period)",
        )
    ]
    * target_cycle
    + ["Discharge at 0.1C until 2.5 V (5 minute period)"],
)

# Run the simulation
sim_k = pybamm.Simulation(model, parameter_values=param, experiment=exp_k, var_pts=var_pts)
sol_k = sim_k.solve()

# Extract total losses
Q_LLI_final_k = sol_k["Total lithium lost [mol]"].entries[-1] * 96485.3 / 3600  # LLI loss (Ah)
Q_LAM_n_k = sol_k["Loss of active material in negative electrode [A.h]"].entries[-1]  # LAMn loss (Ah)
Q_LAM_p_k = sol_k["Loss of active material in positive electrode [A.h]"].entries[-1]  # LAMp loss (Ah)

# Total loss
total_loss_k = Q_LLI_final_k + Q_LAM_n_k + Q_LAM_p_k

# Calculate remaining capacity
initial_capacity = 5  # Ah (adjust if necessary)
remaining_capacity_k = (initial_capacity - total_loss_k) / initial_capacity * 100

print(f"When k={target_cycle}, total loss is {total_loss_k:.2f} Ah")
print(f"When k={target_cycle}, the remaining capacity is: {remaining_capacity_k:.2f}%")


Note: you may need to restart the kernel to use updated packages.
Calculating cycle 1100 ...


At t = 469.698 and h = 9.84043e-12, the corrector convergence failed repeatedly or with |h| = hmin.
At t = 229.698 and h = 1.18976e-11, the corrector convergence failed repeatedly or with |h| = hmin.
At t = 405.257 and h = 1.1382e-11, the corrector convergence failed repeatedly or with |h| = hmin.
At t = 165.257 and h = 2.88891e-12, the corrector convergence failed repeatedly or with |h| = hmin.
At t = 401.022 and h = 2.13515e-14, the corrector convergence failed repeatedly or with |h| = hmin.
At t = 161.025 and h = 1.44557e-11, the corrector convergence failed repeatedly or with |h| = hmin.
At t = 396.826 and h = 3.04688e-12, the corrector convergence failed repeatedly or with |h| = hmin.
At t = 392.788 and h = 8.78111e-12, the corrector convergence failed repeatedly or with |h| = hmin.
At t = 389.036 and h = 1.55725e-11, the corrector convergence failed repeatedly or with |h| = hmin.
At t = 385.538 and h = 4.21543e-12, the corrector convergence failed repeatedly or with |h| = hmin.
A

KeyError: "'Loss of active material in negative electrode [A.h]' not found. Best matches are ['Loss of active material in negative electrode [%]', 'Loss of active material in positive electrode [%]', 'Loss of lithium due to loss of active material in negative electrode [mol]']"